In [41]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.environ["GROQ_API_KEY"]

In [42]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq

In [43]:
llm = ChatGroq(
    model="llama3-8b-8192"
)

In [44]:
llm.invoke("Hello")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.019846164, 'prompt_time': 0.002626493, 'queue_time': None, 'total_time': 0.022472657}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-1a4ca57c-ade2-4fd4-a084-a37d4712adf8-0')

In [45]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2",
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [46]:
os.environ["PINECONE_API_KEY"] = os.environ["PINECONE_API_KEY"]
index_name = "books-pinecone-with-metadata"

vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding
)

In [47]:
vectorstore.similarity_search("Langchain")

[Document(page_content="For Further Exploration\nTo learn more about LangChain and its wide range of applications, be sure to check out the comprehensive documentation and tutorials available on the official website:\n\nLangChain Documentation: https://python.langchain.com/\nDon't Forget to Like and Subscribe!\nIf you're looking for in-depth tutorials and insights into LangChain, CrewAI, and other AI technologies, be sure to check out the fantastic YouTube channel by Brandon Hancock:\n\nYouTube Channel: https://www.youtube.com/@bhancock_ai\nDon't forget to like and subscribe to his channel!!", metadata={'source': 'langchain_demo.txt'}),
 Document(page_content='LangChain: A Framework for LLM-Powered Applications\nLangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimi

In [48]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":2
    }
)

In [49]:
retriever.invoke("Langchain")

[Document(page_content="For Further Exploration\nTo learn more about LangChain and its wide range of applications, be sure to check out the comprehensive documentation and tutorials available on the official website:\n\nLangChain Documentation: https://python.langchain.com/\nDon't Forget to Like and Subscribe!\nIf you're looking for in-depth tutorials and insights into LangChain, CrewAI, and other AI technologies, be sure to check out the fantastic YouTube channel by Brandon Hancock:\n\nYouTube Channel: https://www.youtube.com/@bhancock_ai\nDon't forget to like and subscribe to his channel!!", metadata={'source': 'langchain_demo.txt'}),
 Document(page_content='LangChain: A Framework for LLM-Powered Applications\nLangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimi

In [50]:
# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
    "if asked any normal question answer it."
    # "search about langchain"
)

In [51]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [54]:
history_aware_retriever = create_history_aware_retriever( 
# LLM: to generate search term given chat history.
# retriever: a simple retriever
# contextualize_q_prompt: a simple prompt to guide llm
    llm,retriever,contextualize_q_prompt
)

In [69]:
# testing the history aware retriever

context_test = history_aware_retriever.invoke({
    "chat_history": [
        "What is the capital of France?",
        "Paris"
    ],
    "input": "where is France?"
})

In [76]:
context_test

Document(page_content='We need only confess that we do not know the purpose of the European\nconvulsions and that we know only the facts—that is, the murders, first\nin France, then in Italy, in Africa, in Prussia, in Austria, in Spain,\nand in Russia—and that the movements from the west to the east and from\nthe east to the west form the essence and purpose of these events, and\nnot only shall we have no need to see exceptional ability and genius in\nNapoleon and Alexander, but we shall be unable to consider them to\nbe anything but like other men, and we shall not be obliged to have\nrecourse to chance for an explanation of those small events which made\nthese people what they were, but it will be clear that all those small\nevents were inevitable.', metadata={'source': 'war_and_peace.txt'})

In [56]:
# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}" # This is where the retrieved context will be placed after the histroy aware retriever chain
)

In [57]:
ques_ans_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
        
    ]
)

In [58]:
# Create a chain for passing a list of Documents to a model.


qa_chain = create_stuff_documents_chain(llm,ques_ans_prompt)

In [62]:
qa_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(inp

In [77]:
# creating a retrieval chain that combines the history-aware retriever and the QA chain
# Create retrieval chain that retrieves documents and then passes them on.

rag_chain = create_retrieval_chain(
    history_aware_retriever, # retrieve documents
    qa_chain # get the retrieved documents and pass it to LLM to answer the question
)

In [78]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000258A04A4C50>, search_kwargs={'k': 2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='search about langchain')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(pr

In [81]:
from langchain_core.messages import HumanMessage, AIMessage

def continue_chat():
    print("Start Chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []
    
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        
        # process the user's query through the retrieval chain
        result = rag_chain.invoke(
            {
                "input": query,
                "chat_history": chat_history
            }
        )
        print(f"User: {query}")
        print(f"AI: {result["answer"]}")
        print("\n")
        
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))

In [82]:
if __name__ == "__main__":
    continue_chat()

Start Chatting with the AI! Type 'exit' to end the conversation.
User: what is langchainb
AI: LangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs).


User: who is romeo
AI: In the context of the provided passage, Romeo is a character in William Shakespeare's play "Romeo and Juliet", who is involved in a fight with Tybalt, resulting in the death of Mercutio.


User: who juliet
AI: Juliet is the female protagonist in William Shakespeare's play "Romeo and Juliet", who is a member of the Capulet family and falls in love with Romeo, a Montague, despite their families' hatred for each other.


User: how are they relatated
AI: Romeo and Juliet are romantically involved and are from feuding families, the Montagues and the Capulets, respectively.


User: where is odyssey
AI: The Odyssey is set in ancient Greece, specifically in the city of Troy and the surrounding lands, as well as in va